# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-14-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-14-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-15 05:26:53,33.93911,67.709953,43035,1605,35067,6363.0,Afghanistan,110.549278,3.729522
1,NaN,NaN,NaN,Albania,2020-11-15 05:26:53,41.15330,20.168300,27233,612,12767,13854.0,Albania,946.313156,2.247274
2,NaN,NaN,NaN,Algeria,2020-11-15 05:26:53,28.03390,1.659600,66819,2139,44199,20481.0,Algeria,152.377219,3.201185
3,NaN,NaN,NaN,Andorra,2020-11-15 05:26:53,42.50630,1.521800,5725,75,4675,975.0,Andorra,7409.564486,1.310044
4,NaN,NaN,NaN,Angola,2020-11-15 05:26:53,-11.20270,17.873900,13374,322,6345,6707.0,Angola,40.692177,2.407657


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,41935,41975,42033,42092,42297,42463,42609,42795,42969,43035
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,22721,23210,23705,24206,24731,25294,25801,26211,26701,27233
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,60169,60800,61381,62051,62693,63446,64257,65108,65975,66819


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1554,1554,1556,1558,1574,1577,1581,1591,1595,1605
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,543,549,557,559,571,579,590,598,605,612
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2011,2024,2036,2048,2062,2077,2093,2111,2124,2139


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34440,34440,34446,34458,34721,34954,34967,35024,35036,35067
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,11696,11861,12002,12092,12203,12353,12493,12574,12667,12767
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,41244,41510,41783,42037,42325,42626,42980,42980,43779,44199


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2230,2242,2267,2283,2304,2328,2351,2385,2417,2435
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,7097,7134,7188,7226,7263,7348,7409,7454,7523,7596
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1080,1090,1092,1095,1098,1107,1112,1113,1117,1123


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,31,30,30,30,30,30,32,32,36,36
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,77,83,83,83,83,84,84,84,84,84
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
646,1001.0,Autauga,Alabama,US,2020-11-15 05:26:53,32.539527,-86.644082,2435,36,0,2399.0,"Autauga, Alabama, US",4358.409852,1.478439
683,1075.0,Lamar,Alabama,US,2020-11-15 05:26:53,33.779950,-88.096680,581,7,0,574.0,"Lamar, Alabama, US",4208.620065,1.204819
684,1077.0,Lauderdale,Alabama,US,2020-11-15 05:26:53,34.901719,-87.656247,3259,51,0,3208.0,"Lauderdale, Alabama, US",3514.542376,1.564897


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,10903890,245598,4148444
India,8814579,129635,8205728
Brazil,5848959,165658,5279452


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,10903890,245598,4148444,6509848,2020-11-15 05:26:53,37.936303,-91.379001
India,8814579,129635,8205728,479216,2020-11-15 05:26:53,23.088275,81.806127
Brazil,5848959,165658,5279452,403849,2020-11-15 05:26:53,-12.669522,-48.480493
France,1915713,42600,139760,1733353,2020-11-15 05:26:53,8.314863,-21.695174
Russia,1887836,32536,1415213,440087,2020-11-15 05:26:53,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1052229,19918,0
California,1023067,18253,0
Florida,875096,17489,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1052229,19918,0,1032311,2020-11-15 05:26:53,31.660643,-98.653069
California,1023067,18253,0,1004814,2020-11-15 05:26:53,37.843962,-120.728594
Florida,875096,17489,0,857607,2020-11-15 05:26:53,28.940755,-82.700744
Illinois,562985,11088,0,551897,2020-11-15 05:26:53,39.843618,-89.178330
New York,556551,34010,0,522541,2020-11-15 05:26:53,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,336549,7266,0
Illinois,Cook,246392,5976,0
Florida,Miami-Dade,200259,3707,0
Arizona,Maricopa,174957,3797,0
Texas,Harris,173436,2908,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,336549,7266,0,329283,2020-11-15 05:26:53,34.308284,-118.228241,6037.0
Illinois,Cook,246392,5976,0,240416,2020-11-15 05:26:53,41.841448,-87.816588,17031.0
Florida,Miami-Dade,200259,3707,0,196552,2020-11-15 05:26:53,25.611236,-80.551706,12086.0
Arizona,Maricopa,174957,3797,0,171160,2020-11-15 05:26:53,33.348359,-112.491815,4013.0
Texas,Harris,173436,2908,0,170528,2020-11-15 05:26:53,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,42795,26211,65108,5616,13053,131,1284519,112680,27676,181642,...,3795,69560,1,96140,1253,60784,10,2071,17056,8696
2020-11-13,42969,26701,65975,5725,13228,133,1296378,114383,27682,191228,...,3883,69754,1,96441,1256,61514,10,2072,17093,8765
2020-11-14,43035,27233,66819,5725,13374,134,1304846,115855,27711,198291,...,3957,69987,1,96933,1265,62167,10,2072,17097,8786


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,1591,598,2111,75,315,3,34782,1668,907,1608,...,63,593,0,841,35,542,1,605,350,255
2020-11-13,1595,605,2124,75,317,3,35045,1697,907,1661,...,64,594,0,844,35,548,1,605,350,257
2020-11-14,1605,612,2139,75,322,4,35307,1738,907,1746,...,64,594,0,848,35,558,1,605,350,257


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,35024,12574,42980,4585,6250,124,1100180,70130,25459,107875,...,3103,66814,0,91141,1093,52861,8,1394,16102,8060
2020-11-13,35036,12667,43779,4675,6326,124,1110477,72116,25463,112728,...,3135,66886,0,91452,1101,53367,8,1394,16134,8090
2020-11-14,35067,12767,44199,4675,6345,127,1119366,74105,25467,119415,...,3175,67241,0,91948,1103,53791,8,1394,16137,8096


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2230,2242,2267,2283,2304,2328,2351,2385,2417,2435
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,541,544,553,554,559,565,570,576,579,581
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,2835,2874,2911,2937,2963,3011,3088,3140,3221,3259
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,977,983,995,1003,1007,1014,1028,1037,1050,1071
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6812,6851,6878,6903,6926,6952,6994,7043,7122,7181


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-12,210637,21812,0,266562,128006,1004116,147599,85899,27546,49,...,296725,1031363,143639,2651,1413,198027,123356,30897,309572,20479
2020-11-13,213617,22405,0,269577,130318,1013566,154038,88645,28016,49,...,300458,1043116,145789,2743,1426,199262,125498,31639,318023,21341
2020-11-14,215843,23153,0,273053,132166,1023067,159234,88645,28395,49,...,305120,1052229,151141,2843,1434,200799,127731,32792,323604,21881


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-12        2385    7454    1113  966   2429     669   1062    5345   
2020-11-13        2417    7523    1117  973   2488     673   1068    5429   
2020-11-14        2435    7596    1123  978   2518     675   1075    5470   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-12         1514      870  ...    1042    231     1171      239   
2020-11-13         1545      892  ...    1064    239     1208      252   
2020-11-14         1556      898  ...    1086    248     1225      260   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-12            944  1066   664          0      203    321  
2020-11-13            991  1108   686          0      206    330  
2020-11-14           1024  1156   719          0      208    336  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-12,3213,96,0,6240,2144,18135,2468,4726,732,0,...,3788,19767,687,59,23,3758,2507,555,2621,127
2020-11-13,3231,97,0,6257,2148,18205,2504,4737,734,0,...,3852,19785,701,59,23,3785,2519,565,2683,127
2020-11-14,3246,98,0,6300,2159,18253,2525,4737,736,0,...,3877,19918,710,59,23,3799,2519,574,2739,144


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-12          32      84       9   17     34      19     41      77   
2020-11-13          36      84       9   17     34      19     41      77   
2020-11-14          36      84       9   17     34      19     41      77   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-12           48       15  ...       3      4        8        1   
2020-11-13           48       15  ...       3      4        8        1   
2020-11-14           48       15  ...       4      5        8        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-12              4     2     4          0        7      0  
2020-11-13              4     2     4          0        7      0  
2020-11-14              4     2     4          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,0.004365,0.015891,0.013244,0.008802,0.007720,0.000000,0.008767,0.019286,0.000036,0.053730,...,0.025676,0.002349,0.0,0.004073,0.000799,0.011970,0.0,0.000000,0.001174,0.003346
2020-11-13,0.004066,0.018694,0.013316,0.019409,0.013407,0.015267,0.009232,0.015114,0.000217,0.052774,...,0.023188,0.002789,0.0,0.003131,0.002394,0.012010,0.0,0.000483,0.002169,0.007935
2020-11-14,0.001536,0.019924,0.012793,0.000000,0.011037,0.007519,0.006532,0.012869,0.001048,0.036935,...,0.019057,0.003340,0.0,0.005102,0.007166,0.010615,0.0,0.000000,0.000234,0.002396


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,0.006325,0.013559,0.008600,0.0,0.009615,0.000000,0.007269,0.019560,0.0,0.028133,...,0.000000,0.003384,NaN,0.003580,0.0,0.007435,0.0,0.0,0.0,0.000000
2020-11-13,0.002514,0.011706,0.006158,0.0,0.006349,0.000000,0.007561,0.017386,0.0,0.032960,...,0.015873,0.001686,NaN,0.003567,0.0,0.011070,0.0,0.0,0.0,0.007843
2020-11-14,0.006270,0.011570,0.007062,0.0,0.015773,0.333333,0.007476,0.024160,0.0,0.051174,...,0.000000,0.000000,NaN,0.004739,0.0,0.018248,0.0,0.0,0.0,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,0.001630,0.006484,0.000000,0.021613,0.020408,0.000000,0.016899,0.026253,0.000157,0.039669,...,0.008122,0.002130,NaN,0.004386,0.005520,0.012159,0.0,0.0,0.001991,0.001740
2020-11-13,0.000343,0.007396,0.018590,0.019629,0.012160,0.000000,0.009359,0.028319,0.000157,0.044987,...,0.010313,0.001078,NaN,0.003412,0.007319,0.009572,0.0,0.0,0.001987,0.003722
2020-11-14,0.000885,0.007895,0.009594,0.000000,0.003003,0.024194,0.008005,0.027581,0.000157,0.059320,...,0.012759,0.005308,NaN,0.005424,0.001817,0.007945,0.0,0.0,0.000186,0.000742


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-12,0.009586,0.022549,NaN,0.005276,0.014335,0.008579,0.036495,0.013665,0.007461,0.0,...,0.011398,0.008830,0.028049,0.045759,0.00000,0.007740,0.027872,0.023046,0.027287,0.057035
2020-11-13,0.014148,0.027187,NaN,0.011311,0.018062,0.009411,0.043625,0.031968,0.017062,0.0,...,0.012581,0.011396,0.014968,0.034704,0.00920,0.006237,0.017364,0.024015,0.027299,0.042092
2020-11-14,0.010421,0.033385,NaN,0.012894,0.014181,0.009374,0.033732,0.000000,0.013528,0.0,...,0.015516,0.008736,0.036711,0.036456,0.00561,0.007713,0.017793,0.036442,0.017549,0.025303


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-12      0.014462  0.006074  0.000899  0.005203  0.012083  0.001497   
2020-11-13      0.013417  0.009257  0.003594  0.007246  0.024290  0.005979   
2020-11-14      0.007447  0.009704  0.005372  0.005139  0.012058  0.002972   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-12      0.000000  0.011927  0.003979  0.005780  ...  0.040959   
2020-11-13      0.005650  0.015716  0.020476  0.025287  ...  0.021113   
2020-11-14      0.006554  0.007552  0.007120  0.006726  ...  0.020677   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-12      0.022124  0.065514  0.057522   0.086306  0.040000  0.059011   
2020-11-13      0.034632  0.031597  0.054393   0.049788  0.039400  0.033133   
2020-11-14      0.037657  0.014073  0.031746   0.033300  0.043321  0.048105   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-12            NaN  0.020101 -0.006192  
2020-11-13            NaN  0.014778  0.028037  
2020-11-14            NaN  0.009709  0.018182  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-12,0.003749,0.000000,NaN,0.001927,0.008467,0.001491,0.010233,0.002120,0.011050,NaN,...,0.007179,0.015306,0.013274,0.0,0.0,0.004544,0.010073,0.003617,0.000000,0.000000
2020-11-13,0.005602,0.010417,NaN,0.002724,0.001866,0.003860,0.014587,0.002328,0.002732,NaN,...,0.016895,0.000911,0.020378,0.0,0.0,0.007185,0.004787,0.018018,0.023655,0.000000
2020-11-14,0.004643,0.010309,NaN,0.006872,0.005121,0.002637,0.008387,0.000000,0.002725,NaN,...,0.006490,0.006722,0.012839,0.0,0.0,0.003699,0.000000,0.015929,0.020872,0.133858


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-12       0.000     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-11-13       0.125     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-11-14       0.000     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                           \
Admin2         Chambers Cherokee  ...      Park Platte Sheridan Sublette   
2020-11-12          0.0      0.0  ...  0.000000   0.00      0.0      0.0   
2020-11-13          0.0      0.0  ...  0.000000   0.00      0.0      0.0   
2020-11-14          0.0      0.0  ...  0.333333   0.25      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-12            0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-13            0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-14            0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,0.003930,0.018460,0.012610,0.011134,0.009770,0.000153,0.008537,0.017341,0.000089,0.048352,...,0.022548,0.003044,0.0,0.003592,0.007009,0.011215,4.982395e-45,0.000068,0.001637,0.004475
2020-11-13,0.003998,0.018577,0.012963,0.015271,0.011588,0.007710,0.008885,0.016227,0.000153,0.050563,...,0.022868,0.002917,0.0,0.003361,0.004702,0.011612,2.491197e-45,0.000275,0.001903,0.006205
2020-11-14,0.002767,0.019251,0.012878,0.007636,0.011313,0.007614,0.007708,0.014548,0.000600,0.043749,...,0.020963,0.003128,0.0,0.004232,0.005934,0.011114,1.245599e-45,0.000138,0.001069,0.004300


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,0.004761,0.015139,0.007936,2.310801e-06,0.008943,1.531160e-64,0.008312,0.018240,1.687995e-08,0.032566,...,0.004576,0.003590,NaN,0.004090,1.684337e-23,0.009998,0.0,0.000326,0.000358,0.001138
2020-11-13,0.003638,0.013422,0.007047,1.155400e-06,0.007646,7.655801e-65,0.007937,0.017813,8.439975e-09,0.032763,...,0.010224,0.002638,NaN,0.003829,8.421685e-24,0.010534,0.0,0.000163,0.000179,0.004491
2020-11-14,0.004954,0.012496,0.007055,5.777002e-07,0.011710,1.666667e-01,0.007706,0.020987,4.219988e-09,0.041968,...,0.005112,0.001319,NaN,0.004284,4.210842e-24,0.014391,0.0,0.000081,0.000090,0.002245


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-12,0.002248,0.008791,0.003785,0.021421,0.018494,0.004647,0.010676,0.024363,0.000111,0.042753,...,0.011409,0.003372,NaN,0.004115,0.003765,0.012526,2.919372e-47,0.000867,0.002285,0.001545
2020-11-13,0.001295,0.008094,0.011187,0.020525,0.015327,0.002324,0.010018,0.026341,0.000134,0.043870,...,0.010861,0.002225,NaN,0.003764,0.005542,0.011049,1.459686e-47,0.000433,0.002136,0.002634
2020-11-14,0.001090,0.007994,0.010391,0.010262,0.009165,0.013259,0.009011,0.026961,0.000145,0.051595,...,0.011810,0.003766,NaN,0.004594,0.003679,0.009497,7.298430e-48,0.000217,0.001161,0.001688


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-12,0.009143,0.023767,NaN,0.006826,0.013741,0.007735,0.032341,0.017444,0.008199,9.633701e-74,...,0.011812,0.009817,0.022940,0.034250,0.001073,0.007772,0.016908,0.023673,0.025598,0.043554
2020-11-13,0.011645,0.025477,NaN,0.009068,0.015901,0.008573,0.037983,0.024706,0.012631,4.816851e-74,...,0.012196,0.010606,0.018954,0.034477,0.005136,0.007004,0.017136,0.023844,0.026449,0.042823
2020-11-14,0.011033,0.029431,NaN,0.010981,0.015041,0.008973,0.035858,0.012353,0.013079,2.408425e-74,...,0.013856,0.009671,0.027832,0.035467,0.005373,0.007359,0.017465,0.030143,0.021999,0.034063


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-12      0.012071  0.007264  0.002985  0.008926  0.012297  0.002169   
2020-11-13      0.012744  0.008260  0.003290  0.008086  0.018293  0.004074   
2020-11-14      0.010096  0.008982  0.004331  0.006613  0.015176  0.003523   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-12      0.001472  0.009142  0.004155  0.007707  ...  0.030202   
2020-11-13      0.003561  0.012429  0.012315  0.016497  ...  0.025658   
2020-11-14      0.005058  0.009990  0.009717  0.011612  ...  0.023167   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-12      0.025758  0.043973  0.039050   0.066988  0.026587  0.047661   
2020-11-13      0.030195  0.037785  0.046722   0.058388  0.032993  0.040397   
2020-11-14      0.033926  0.025929  0.039234   0.045844  0.038157  0.044251   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-12           -1.0  0.018908  0.008847  
2020-11-13           -1.0  0.016843  0.018442  
2020-11-14           -1.0  0.013276  0.018312  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-12,0.010118,0.022791,NaN,0.003235,0.007304,0.001907,0.008406,0.002181,0.006941,NaN,...,0.012464,0.010480,0.011688,0.000269,0.000135,0.003944,0.006767,0.012739,0.002840,0.015866
2020-11-13,0.007860,0.016604,NaN,0.002980,0.004585,0.002883,0.011496,0.002254,0.004837,NaN,...,0.014680,0.005695,0.016033,0.000135,0.000068,0.005564,0.005777,0.015378,0.013248,0.007933
2020-11-14,0.006251,0.013457,NaN,0.004926,0.004853,0.002760,0.009942,0.001127,0.003781,NaN,...,0.010585,0.006209,0.014436,0.000067,0.000034,0.004632,0.002888,0.015654,0.017060,0.070896


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-11-12      0.016415  0.002237  4.157690e-12  0.008855  0.077236   
2020-11-13      0.070707  0.001119  2.078845e-12  0.004427  0.038618   
2020-11-14      0.035354  0.000559  1.039423e-12  0.002214  0.019309   

Province_State                                                        ...  \
Admin2           Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-12      0.021242  1.907349e-07  0.035137  0.000042  0.000004  ...   
2020-11-13      0.010621  9.536743e-08  0.017569  0.000021  0.000002  ...   
2020-11-14      0.005310  4.768372e-08  0.008784  0.000011  0.000001  ...   

Province_State       Wyoming                                          \
Admin2                  Park    Platte  Sheridan Sublette Sweetwater   
2020-11-12      2.980232e-08  0.001143  0.010734      0.0   0.001953   
2020-11-13      1.490116e-08  0.000572  0.005367      0.0   0.000977   
2020-11-14      1.666667e-01  0.125286  0.002684      0.0   0.000488   

Province_State                                                         
Admin2             Teton     Uinta Unassigned      Washakie    Weston  
2020-11-12      0.000977  0.010417  -0.972376  7.761021e-11 -0.533333  
2020-11-13      0.000488  0.005208  -0.972376  3.880511e-11 -0.533333  
2020-11-14      0.000244  0.002604  -0.972376  1.940255e-11 -0.533333  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201114,AK,22405,NaN,839859,NaN,totalTestsViral,862264,125.0,NaN,...,862264,2,0,444c355ea5e0d86f51f91fa40083c77d2363a115,0,0,0,0,0,NaN
1,20201114,AL,215843,33948.0,1271260,NaN,totalTestsViral,1453155,1120.0,22275.0,...,1487103,15,0,1063ee1523d64fc0d16f9e585811ade9bc1de18b,0,0,0,0,0,NaN
2,20201114,AR,130318,13531.0,1367077,NaN,totalTestsViral,1483864,799.0,7847.0,...,1497395,0,30,2716c9f7fc491cc9cba49944f1bd51db338b5fdd,0,0,0,0,0,NaN
3,20201114,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,...,1768,0,0,cee0c67a3517a0c5fc7e7c18571f82ae7ec7510c,0,0,0,0,0,NaN
4,20201114,AZ,273053,7245.0,1695606,NaN,totalTestsPeopleViral,1961414,1470.0,22937.0,...,1968659,43,117,e11548e63ec5404c5a2f79bd29224bb080a063d5,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201114,AK,22405.0,NaN,839859.0,NaN,totalTestsViral,862264.0,125.0,NaN,...,862264,2,0,444c355ea5e0d86f51f91fa40083c77d2363a115,0,0,0,0,0,NaN
1,20201114,AL,215843.0,33948.0,1271260.0,NaN,totalTestsViral,1453155.0,1120.0,22275.0,...,1487103,15,0,1063ee1523d64fc0d16f9e585811ade9bc1de18b,0,0,0,0,0,NaN
2,20201114,AR,130318.0,13531.0,1367077.0,NaN,totalTestsViral,1483864.0,799.0,7847.0,...,1497395,0,30,2716c9f7fc491cc9cba49944f1bd51db338b5fdd,0,0,0,0,0,NaN
3,20201114,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,...,1768,0,0,cee0c67a3517a0c5fc7e7c18571f82ae7ec7510c,0,0,0,0,0,NaN
4,20201114,AZ,273053.0,7245.0,1695606.0,NaN,totalTestsPeopleViral,1961414.0,1470.0,22937.0,...,1968659,43,117,e11548e63ec5404c5a2f79bd29224bb080a063d5,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-14,AK,22405,NaN,839859,NaN,totalTestsViral,862264,125.0,NaN,NaN,...,862264,2,0,444c355ea5e0d86f51f91fa40083c77d2363a115,0,0,0,0,0,NaN
2020-11-14,AL,215843,33948.0,1271260,NaN,totalTestsViral,1453155,1120.0,22275.0,NaN,...,1487103,15,0,1063ee1523d64fc0d16f9e585811ade9bc1de18b,0,0,0,0,0,NaN
2020-11-14,AR,130318,13531.0,1367077,NaN,totalTestsViral,1483864,799.0,7847.0,291.0,...,1497395,0,30,2716c9f7fc491cc9cba49944f1bd51db338b5fdd,0,0,0,0,0,NaN
2020-11-14,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,NaN,...,1768,0,0,cee0c67a3517a0c5fc7e7c18571f82ae7ec7510c,0,0,0,0,0,NaN
2020-11-14,AZ,273053,7245.0,1695606,NaN,totalTestsPeopleViral,1961414,1470.0,22937.0,362.0,...,1968659,43,117,e11548e63ec5404c5a2f79bd29224bb080a063d5,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-14,AK,22405.0,NaN,839859.0,NaN,totalTestsViral,862264.0,125.0,NaN,NaN,...,862264,2,0,444c355ea5e0d86f51f91fa40083c77d2363a115,0,0,0,0,0,NaN
2020-11-14,AL,215843.0,33948.0,1271260.0,NaN,totalTestsViral,1453155.0,1120.0,22275.0,NaN,...,1487103,15,0,1063ee1523d64fc0d16f9e585811ade9bc1de18b,0,0,0,0,0,NaN
2020-11-14,AR,130318.0,13531.0,1367077.0,NaN,totalTestsViral,1483864.0,799.0,7847.0,291.0,...,1497395,0,30,2716c9f7fc491cc9cba49944f1bd51db338b5fdd,0,0,0,0,0,NaN
2020-11-14,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,NaN,...,1768,0,0,cee0c67a3517a0c5fc7e7c18571f82ae7ec7510c,0,0,0,0,0,NaN
2020-11-14,AZ,273053.0,7245.0,1695606.0,NaN,totalTestsPeopleViral,1961414.0,1470.0,22937.0,362.0,...,1968659,43,117,e11548e63ec5404c5a2f79bd29224bb080a063d5,0,0,0,0,0,NaN
